# Xarray and NetCDF
import libraries

In [1]:
import os
import urllib
import pandas as pd
import numpy  as np
import xarray as xr

## Data

In [19]:
temp_data = np.array([np.zeros((5,5)),
                      np.ones((5,5)),
                      np.ones((5,5))*2
                      ]).astype(int)
temp_data

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])

## Dimensions

In [23]:
dims   = ('time', 'lat', 'lon')
coords = {'time': pd.date_range('2022-09-01', '2022-09-03'),
          'lat' : np.arange(70, 20, -10),
          'lon': np.arange(60, 110, 10)
          }

## Attributes

In [24]:
attrs = {'title': 'temperature across data array'}

## Generate Xarray

In [25]:
temp = xr.DataArray(data = temp_data,
                    dims = dims, 
                    coords = coords, 
                    attrs = attrs)
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)>
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:    temperature across data array

## Add attributes to existing Xarray Data

In [26]:
temp.attrs['Description'] = 'simple example of xarray'
temp.lat.attrs['units']   = 'degree_n'
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)>
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:        temperature across data array
    Description:  simple example of xarray

## Working with data
  - Indexing
  - Aggregating

In [27]:
temp[1, 1, 2]

<xarray.DataArray ()>
array(1)
Coordinates:
    time     datetime64[ns] 2022-09-02
    lat      int64 60
    lon      int64 80
Attributes:
    title:        temperature across data array
    Description:  simple example of xarray

In [28]:
#temp.sel(time = '2022-09-01', lat = 40, lon = 80)
temp.sel(time='2022-09-01', lat=40, lon=80)

<xarray.DataArray ()>
array(0)
Coordinates:
    time     datetime64[ns] 2022-09-01
    lat      int64 40
    lon      int64 80
Attributes:
    title:        temperature across data array
    Description:  simple example of xarray

In [29]:
avg_temp = temp.mean(dim = 'time', keep_attrs = True)
avg_temp

<xarray.DataArray (lat: 5, lon: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:        temperature across data array
    Description:  simple example of xarray

In [31]:
temp.sel(time = '2022-09-01', lon = slice(40, 80, 2))

<xarray.DataArray (lat: 5, lon: 2)>
array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])
Coordinates:
    time     datetime64[ns] 2022-09-01
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 80
Attributes:
    title:        temperature across data array
    Description:  simple example of xarray

In [33]:
data_var = {'avg_temp': avg_temp, 
            'temp':    temp}
attrs = {'title' : 'temperature data at weather stations with daily and average temps'}

temp_dataset = xr.Dataset(data_vars = data_var,
                          attrs = attrs)

In [35]:
temp_dataset.to_netcdf("temp_example.nc")

In [36]:
check = xr.open_dataset('temp_example.nc')
check

<xarray.Dataset>
Dimensions:   (lat: 5, lon: 5, time: 3)
Coordinates:
  * lat       (lat) int64 70 60 50 40 30
  * lon       (lon) int64 60 70 80 90 100
  * time      (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
Data variables:
    avg_temp  (lat, lon) float64 ...
    temp      (time, lat, lon) int64 ...
Attributes:
    title:    temperature data at weather stations with daily and average temps

In [37]:
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A792bfc37-416e-409e-80b1-fdef8ab60033'

msg = urllib.request.urlretrieve(url, "FW_data_CESM_LW_2006_2100.nc")

In [38]:
fp = os.path.join(os.getcwd(),'FW_data_CESM_LW_2006_2100.nc')
fw_data = xr.open_dataset(fp)
fw_data

<xarray.Dataset>
Dimensions:                          (time: 95, member: 11)
Coordinates:
  * time                             (time) float64 2.006e+03 ... 2.1e+03
  * member                           (member) float64 1.0 2.0 3.0 ... 10.0 11.0
Data variables: (12/16)
    FW_flux_Fram_annual_net          (time, member) float64 ...
    FW_flux_Barrow_annual_net        (time, member) float64 ...
    FW_flux_Nares_annual_net         (time, member) float64 ...
    FW_flux_Davis_annual_net         (time, member) float64 ...
    FW_flux_BSO_annual_net           (time, member) float64 ...
    FW_flux_Bering_annual_net        (time, member) float64 ...
    ...                               ...
    Solid_FW_flux_BSO_annual_net     (time, member) float64 ...
    Solid_FW_flux_Bering_annual_net  (time, member) float64 ...
    runoff_annual                    (time, member) float64 ...
    netPrec_annual                   (time, member) float64 ...
    Liquid_FW_storage_Arctic_annual  (time, member) float64 ...
    Solid_FW_storage_Arctic_annual   (time, member) float64 ...
Attributes:
    creation_date:   02-Jun-2020 15:38:31
    author:          Alexandra Jahn, CU Boulder, alexandra.jahn@colorado.edu
    title:           Annual timeseries of freshwater data from the CESM Low W...
    description:     Annual mean Freshwater (FW) fluxes and storage relative ...
    data_structure:  The data structure is |Ensemble member | Time (in years)...

## Select values for the second ensemble member of the netPrec_annual variable
Note that we call out the variable in fw_data using the dot syntax (dataset.variable)

In [41]:
member2 = fw_data.netPrec_annual.sel(member = 2 )
member2

<xarray.DataArray 'netPrec_annual' (time: 95)>
array([2208.638737, 2039.416637, 2047.057141, 2025.87982 , 1822.187912,
       1669.356201, 2158.327121, 1953.640577, 1870.981984, 1772.145359,
       1999.080656, 2018.159627, 2001.870776, 1879.494624, 1763.844963,
       2041.952846, 1945.355813, 1864.957608, 2255.011079, 1908.527127,
       1871.596426, 2066.698113, 1915.627194, 1606.202663, 2163.072721,
       1999.894225, 2128.807793, 1730.157807, 2004.363711, 1934.619578,
       1961.641825, 2133.984507, 2100.872032, 1998.152868, 1936.160188,
       1888.979096, 1739.643399, 2183.405479, 2234.175433, 1913.12681 ,
       1844.599277, 1782.303602, 1988.65111 , 2110.562963, 1804.745419,
       2431.100324, 2278.126158, 2026.546473, 2086.390118, 2039.196003,
       2212.42824 , 2124.793176, 1717.164921, 2281.361115, 1968.934899,
       1898.694923, 1897.966857, 2256.511726, 2142.77437 , 2150.77564 ,
       2059.183773, 1954.013644, 1851.009509, 2029.365686, 1909.682408,
       2097.263358, 1959.62086 , 2080.566397, 1983.871736, 1884.143938,
       2104.651123, 1763.933545, 2018.07668 , 1985.591128, 2114.668759,
       2229.904322, 2061.211734, 1991.483719, 2145.538448, 1856.963699,
       2092.508225, 2125.400212, 2119.526455, 2159.713051, 1876.473441,
       2169.930131, 2224.98958 , 2070.136476, 1976.527478, 2210.090693,
       2036.273118, 2044.633842, 1946.276296, 1961.74668 , 1904.765559])
Coordinates:
  * time     (time) float64 2.006e+03 2.007e+03 2.008e+03 ... 2.099e+03 2.1e+03
    member   float64 2.0
Attributes:
    units:      km3/yr
    long_name:  Net FW flux from precipitation minus evaporation over the Arc...

What is the max value of the second member or the'netPrec_annual' variable between 2022 and 2100?

In [53]:

sub_fw = fw_data.netPrec_annual.sel(member = 2, time = slice(2022,2100,1) ).max().item()
sub_fw2 = max(fw_data.netPrec_annual.sel(member = 2, time = slice(2022,2100,1))).item()
#sub_fw
sub_fw2

2431.100323507693